In [53]:
from collections import OrderedDict
def uint32(v): return (v & 0xFFFFFFFF)
def uint40(v): return (v & 0xFFFFFFFFFF)

def warn(msg):
    print("WARNING", msg)

class Register:
    def __init__(self, number, hardware_name, assembly_name, bits):
        self.number = number
        self.name = hardware_name
        self.hardware_name = hardware_name
        self.assembly_name = assembly_name
        self.bits = bits
        self.data = [0] * (bits // 8)

    def __str__(self):
        if self.number >= 0:
            r = f"{self.assembly_name} [{self.hardware_name}={self.number}]"
        elif self.hardware_name != self.assembly_name and self.assembly_name is not None:
            r = f"{self.assembly_name} [{self.hardware_name}]"
        else:
            r = self.hardware_name
        return r + " " + str(self.data)
        
    @property
    def int(self):
        v = self.uint
        if v & 0x80000000:
            v &= 0x7FFFFFFF
            v -= 0x80000000
        return v

    @int.setter
    def int(self, value):
        if value < -(1<<self.bits-1):
            warn("value < min when setting uint32")
        if value >= 1 << (self.bits-1):
            warn("value > max when setting uint32")
        for i in range(len(self.data)):
            self.data[i] = (value >> (i<<3)) & 0xFF
            
        v = value 
    
    @property
    def uint(self):
        return sum(d << (i << 3) for i,d in enumerate(self.data)) 

    @uint.setter
    def uint(self, value):
        if value < 0:
            warn("value < min when setting uint32")
        if value >= 1 << self.bits:
            warn("value > max when setting uint32")
        for i in range(len(self.data)):
            self.data[i] = (value >> (i<<3)) & 0xFF
            
class InstructionFormat(OrderedDict):
    def __init__(self, name="---"):
        if name == "RRR": super().__init__(dict(op2=4, op1=4, r=4, s=4, t=4, op0=4))
        elif name == "RRI4": super().__init__(dict(imm4=4, op1=4, r=4, s=4, t=4, op0=4))
        elif name == "RRI8": super().__init__(dict(imm8=8, r=4, s=4, t=4, op0=4))
        elif name == "RI16": super().__init__(dict(imm16=16, t=4, op0=4))
        elif name == "RSR": super().__init__(dict(op2=4, op1=4, rs=8, t=4, op0=4))
        elif name == "CALL": super().__init__(dict(ofset=18, n=2, op0=4))
        elif name == "CALLX": super().__init__(dict(op2=4, op1=4, r=4, s=4, m=2, n=2, op0=4))
        elif name == "BRI8": super().__init__(dict(imm8=8, r=4, s=4, m=2, n=2, op0=4))
        elif name == "BRI12": super().__init__(dict(imm12=12, s=4, m=2, n=2, op0=4))
        elif name == "RRRN": super().__init__(dict(r=4, s=4, t=4, op0=4))
        elif name == "RI7": super().__init__(dict(imm7_l=4,s=4, i_imm7_h=4, op0=4))
        elif name == "RI6": super().__init__(dict(imm6_l=4, s=4, i_z_imm6_h=4, op0=4))
        else: super().__init__(dict(op0=24))
        self.name = name

class InstructionDescriptor(object):
    def __init__(self, template, format, description, cb=None):
        self.template = template
        self.name, *argument_names = template.split(" ",1)
        self.argument_names = argument_names[0].replace(" ","").split(",") if argument_names else []
        self.format = format
        self.description = description
        self.callback = cb

    def __str__(self):
        return "[" + ", ".join(self.argument_names) + "] " + self.description

class Instruction(object):
    def __init__(self, context, descriptor, arguments):
        self.context = context
        self.descriptor = descriptor
        self.arguments = arguments

    @classmethod
    def parse(cls, context, line):
        name, arguments = line.split(" ",1)
        arguments = arguments.replace(" ","").split(",")
        descriptor = context.find_instruction(name)
        return cls(context, descriptor, arguments)

    def __str__(self):
        line = " ".join((self.descriptor.name, ", ".join(self.arguments)))
        return line + (" " * (32-len(line)) if len(line) < 32 else " ") + "# " + str(self.descriptor)
        
class InstructionSet(object):
    def __init__(self, name=""):
        self.name = name
        self.parent = None
        self.registers = {}
        self.instructions = {}
        self.expansions = {}
        self.endianness = "le"
        self.options = {}

    def find_instruction(self, instruction):
        while isinstance(instruction,str):
            name = instruction
            if name not in self.instructions and self.parent is not None:
                return self.parent.find_instruction(name)
            instruction = self.instructions[name]
            if isinstance(instruction, InstructionSetOption):
                return instruction.find_instruction(name)
        assert isinstance(instruction, InstructionDescriptor)
        return instruction


    def find_register(self, register):
        while isinstance(register, str):
            name = register
            if name not in self.registers and self.parent is not None:
                return self.parent.find_register(name)
            register = self.registers[name]
            if isinstance(register, InstructionSetOption):
                return register.find_register(name)
        assert isinstance(register, Register)
        return register
        
    def extend(self, option):
        option.parent = self
        self.options[option.name] = option
        
        for r in option.registers:
            if r in self.registers:
                print(f"WARNING. {option} overwrites register {r}")
            self.registers[r] = option #.registers[r]
        for i in option.instructions:
            if i in self.instructions:
                print(f"WARNING. {option} overwrites instruction {i}")
            self.instructions[i] = option #.instructions[i]

        
class InstructionSetOption(InstructionSet): ...
class CoreInstructionSet(InstructionSet): 
    def __init__(self):
        super().__init__("core")
        self.registers = {
            "AR0": Register(0, "AR0", "a0", 32),
            "AR1": Register(1, "AR1", "a1", 32),
            "AR2": Register(2, "AR0", "a2", 32),
            "AR3": Register(3, "AR1", "a3", 32),
            "AR4": Register(4, "AR0", "a4", 32),
            "AR5": Register(5, "AR1", "a5", 32),
            "AR6": Register(6, "AR0", "a6", 32),
            "AR7": Register(7, "AR1", "a7", 32),
            "AR8": Register(8, "AR0", "a8", 32),
            "AR9": Register(9, "AR9", "a9", 32),
            "AR10": Register(10, "AR10", "a10", 32),
            "AR11": Register(11, "AR11", "a11", 32),
            "AR12": Register(12, "AR12", "a12", 32),
            "AR13": Register(13, "AR13", "a13", 32),
            "AR14": Register(14, "AR14", "a14", 32),
            "AR15": Register(15, "AR15", "a15", 32),
            "PC": Register(-1, "PC", "PC", 32),
            "SAR": Register(-1, "SAR","SAR", 8),    
        }

        self.instructions = {
            "L8UI": InstructionDescriptor("L8UI at, as, 0..255", InstructionFormat("RRI8"), "8-bit unsigned load (8-bit offset)"),
            "L16SI": InstructionDescriptor("L16SI at, as, 0..510", InstructionFormat("RRI8"), "16-bit signed load (8-bit shifted offset)"),
            "L16UI": InstructionDescriptor("L16UI at, as, 0..510", InstructionFormat("RRI8"), "16-bit unsigned load (8-bit shifted offset)"),
            "L32I": InstructionDescriptor("L32I at, as, 0..1020", InstructionFormat("RRI8"), "32-bit load (8-bit shifted offset)"),
            "L32R": InstructionDescriptor("L32R at, label", InstructionFormat("RI16"), "32-bit load PC-relative (16-bit negative word offset)"),
            "S8I": InstructionDescriptor("S8I at, as, 0..255", InstructionFormat("RRI8"), "8-bit store (8-bit offset)"),
            "S16I": InstructionDescriptor("S16I at, as, 0..510", InstructionFormat("RRI8"), "16-bit store (8-bit shifted offset)"),
            "S32I": InstructionDescriptor("S32I at, as, 0..1020", InstructionFormat("RRI8"), "32-bit store (8-bit shifted offset)"),
            "MEMW": InstructionDescriptor("MEMW", InstructionFormat("RRR"), "Order memory accesses before with memory access after"),
            "EXTW": InstructionDescriptor("EXTW", InstructionFormat("RRR"), "Order all external effects before with all external effects after"),
            "CALLO": InstructionDescriptor("CALL0 label", InstructionFormat("CALL"), "Call subroutine, PC-relative"),
            "CALLX0": InstructionDescriptor("CALLX0 label", InstructionFormat("CALLX"), "Call subroutine, address in register"),
            "RET": InstructionDescriptor("RET", InstructionFormat("CALLX"), "Subroutine return -- jump to return address"),
            "J": InstructionDescriptor("J label", InstructionFormat("CALLX"),"Unconditional jump, PC-relative"),
            "JX": InstructionDescriptor("JX as", InstructionFormat("CALLX"), "Unconditional jump, address in register"),
            "BALL": InstructionDescriptor("BALL as, at, label", InstructionFormat("RRI8"), "Branch if all of the masked bits are set"),
            "BNALL": InstructionDescriptor("BNALL as, at, label", InstructionFormat("RRI8"),"Branch if not all of the masked bits are set"),
            "BANY": InstructionDescriptor("BANY as, at, label", InstructionFormat("RRI8"), "Branch if any of the masked bits are set"),
            "BNONE": InstructionDescriptor("BNONE as, at, label", InstructionFormat("RRI8"), "Branch if none of the masked bits are set (All Clear)"),
            "BBC": InstructionDescriptor("BBC as, at, label", InstructionFormat("RRI8"), "Branch if bit clear"),
            "BBCI": InstructionDescriptor("BBCI as, 0..31, label", InstructionFormat("RRI8"), "Branch if bit clear immediate"),
            "BBS": InstructionDescriptor("BBS as, at, label", InstructionFormat("RRI8"), "Branch if bit set"),
            "BBSI": InstructionDescriptor("BBSI as, 0..31, label", InstructionFormat("RRI8"), "Branch is bit set immediate"),
            "BEQ": InstructionDescriptor("BEQ as, at, label", InstructionFormat("RRI8"), "Branch if equal"),
            "BEQI": InstructionDescriptor("BEQI as, imm, label", InstructionFormat("BRI8"), "Branch if equal immediate"),
            "BEQZ": InstructionDescriptor("BEQZ as, label", InstructionFormat("BRI12"), "Branch is equal to zero"),
            "BNE": InstructionDescriptor("BNE as, at, label", InstructionFormat("RRI8"), "Branch if not equal"),
            "BNEI": InstructionDescriptor("BNEI as, imm, label", InstructionFormat("BRI8"), "Branch if not equal to immediate"),
            "BNEZ": InstructionDescriptor("BNEZ as, label", InstructionFormat("BRI12"), "Branch if not equal to zero"),
            "BGE": InstructionDescriptor("BGE as, at, label", InstructionFormat("RRI8"), "Branch if greater than or equal"),
            "BGEI": InstructionDescriptor("BGEI as, imm, label", InstructionFormat("BRI8"),"Branch if greater than or equal to immediate"),
            "BGEU": InstructionDescriptor("BGEU as, at, label", InstructionFormat("RRI8"), "Branch if greater than or equal unsigned"),
            "BGEUI": InstructionDescriptor("BGEUI as, imm, label", InstructionFormat("BRI8"), "Branch if greater than or equal to unsigned immediate"),
            "BGEZ": InstructionDescriptor("BGEZ as, label", InstructionFormat("BRI12"), "Branch to greater than or equal to zero"),
            "BLT": InstructionDescriptor("BLT as, at, label", InstructionFormat("RRI8"),"Branch if less than"),
            "BLTI": InstructionDescriptor("BLTI as, imm, label", InstructionFormat("BRI8"), "Branch if less than immediate"),
            "BLTU": InstructionDescriptor("BLTU as, at, label", InstructionFormat("RRI8"), "Branch if less than unsigned"),
            "BLTUI": InstructionDescriptor("BLTUI as, imm, label", InstructionFormat("BRI8"), "Branch if less than unsigned immediate"),
            "BLTZ": InstructionDescriptor("BLTZ as, label", InstructionFormat("RRI8"), "Branch if less than zero"),
            "MOVI": InstructionDescriptor("MOVI at, -2048..2047", InstructionFormat("RRI8"), "Load register with 12-bit signed constant"),
            "MOVEQZ": InstructionDescriptor("MOVEQZ ar, as, at", InstructionFormat("RRR"), "Conditional move if zero"),
            "MOVNEZ": InstructionDescriptor("MOVNEZ ar, as, at", InstructionFormat("RRR"), "Conditional move if non-zero"),
            "MOVLTZ": InstructionDescriptor("MOVLTZ ar, as, at", InstructionFormat("RRR"), "Conditional move if less than zero"),
            "MOVGEZ": InstructionDescriptor("MOVGEZ ar, as, at", InstructionFormat("RRR"), "Conditional move if greater than or equal to zero"),
            "ADD": InstructionDescriptor("ADD ar, as, at", InstructionFormat("RRR"), "Add two registers"),
            "ADDI": InstructionDescriptor("ADDI at, as, -128..127", InstructionFormat("RRI8"), "Add signed constant to register"),
            "ADDMI": InstructionDescriptor("ADDMI at, as, -32768..32512", InstructionFormat("RRI8"), "Add signed constant shifted by 8 to register"),
            "ADDX2": InstructionDescriptor("ADDX2 ar, as, at", InstructionFormat("RRR"), "Add register to register shifted by 1"),
            "ADDX4": InstructionDescriptor("ADDX4 ar, as, at", InstructionFormat("RRR"), "Add register to register shifted by 2"),
            "ADDX8": InstructionDescriptor("ADDX8 ar, as, at", InstructionFormat("RRR"), "Add register to register shifted by 3"),
            "SUB": InstructionDescriptor("SUB ar, as, at", InstructionFormat("RRR"), "Subtract two registers"),
            "SUBX2": InstructionDescriptor("SUBX2 ar, as, at", InstructionFormat("RRR"), "Subtract register from register shifted by 1"),
            "SUBX4": InstructionDescriptor("SUBX4 ar, as, at", InstructionFormat("RRR"), "Subtract register from register shifted by 2"),
            "SUBX8": InstructionDescriptor("SUBX8 ar, as, at", InstructionFormat("RRR"), "Subtract register from register shifted by 3"),
            "NEG": InstructionDescriptor("NEG ar, at", InstructionFormat("RRR"), "Negate"),
            "ABS": InstructionDescriptor("ABS ar, at", InstructionFormat("RRR"), "Absolute value"),
            "SALT": InstructionDescriptor("SALT ar, as, at", InstructionFormat("RRR"), "Set AR if less than"),
            "SALTU": InstructionDescriptor("SALTU ar, as, at", InstructionFormat("RRR"), "Set AR if less than unsigned"),
            "AND": InstructionDescriptor("AND ar, as, at", InstructionFormat("RRR"), "Bitwise logical and"),
            "OR": InstructionDescriptor("OR ar, as, at", InstructionFormat("RRR"), "Bitwise logical or"),
            "XOR": InstructionDescriptor("XOR ar, as, at", InstructionFormat("RRR"), "Bitwise logical xor"),
            "EXTUI": InstructionDescriptor("EXTUI ar, at, shiftimm, maskimm", InstructionFormat("RRR"), "Extract unsigned field immediate"),
            "SRLI": InstructionDescriptor("SRLI ar, at, 0..15", InstructionFormat("RRR"), "Shift right logical immediate"),
            "SRAI": InstructionDescriptor("SRAI ar, at, 0..15", InstructionFormat("RRR"), "Shift right arithmetic immediate"),
            "SLLI": InstructionDescriptor("SLLI ar, at, 0..15", InstructionFormat("RRR"), "Shift left logical immediate"),
            "SRC": InstructionDescriptor("SRC ar, as, at", InstructionFormat("RRR"), "Shift right combined"),
            "SLL": InstructionDescriptor("SLL ar, at", InstructionFormat("RRR"), "Shift left logical"),
            "SRL": InstructionDescriptor("SRL ar, at", InstructionFormat("RRR"), "Shift right logical"),
            "SRA": InstructionDescriptor("SRA ar, at", InstructionFormat("RRR"), "Shift right arithmetic"),
            "SSL": InstructionDescriptor("SSL as", InstructionFormat("RRR"), "Set shift amount register for shift left logical"),
            "SSR": InstructionDescriptor("SSR as", InstructionFormat("RRR"), "Set shift amount register for shift right logical"),
            "SSAI":InstructionDescriptor("SRAI ar, at, 0..31", InstructionFormat("RRR"), "Set shift amount register immediate"),
            "SSA8B":InstructionDescriptor("SSA8B as", InstructionFormat("RRR"), "Set shift amount register for big-endian byte align"),
            "SSA8L": InstructionDescriptor("SSA8L as", InstructionFormat("RRR"), "Set shift amount register for little-endian byte align"),
            "RSR": InstructionDescriptor("RSR ar, sr", InstructionFormat("RSR"), "Read special register"),
            "WSR": InstructionDescriptor("WSR at, sr", InstructionFormat("RSR"), "Write special register"),
            "XSR": InstructionDescriptor("XSR at, sr", InstructionFormat("RSR"), "Exchange special register"),
            "RUR": InstructionDescriptor("RUR ar, sr", InstructionFormat("RUR"), "Read user defined register"),
            "WUR": InstructionDescriptor("WUR at, sr", InstructionFormat("RUR"), "Write user defined register"),
            "ISYNC": InstructionDescriptor("ISYNC", InstructionFormat("RRR"), "Instruction fetch synchronize"),
            "RSYNC": InstructionDescriptor("RSYNC", InstructionFormat("RRR"), "Instruction register synchronize"),
            "ESYNC": InstructionDescriptor("ESYNC", InstructionFormat("RRR"), "Register value synchronize"),
            "DSYNC": InstructionDescriptor("DSYNC", InstructionFormat("RRR"), "Load/store synchronize"),
            "FSYNC": InstructionDescriptor("FSYNC", InstructionFormat("RRR"), "Fetch synchronize"),
            "NOP": InstructionDescriptor("NOP", InstructionFormat("RRR"), "No operation"),
        }

        self.b4const = [-1,1,2,3,4,5,6,7,8,10,12,16,32,64,128,256]
        self.b4constu = [32768, 65536, 2,3,4,5,6,7,8,10,12,16,32,64,128,256]
        
        self.memory = [0] * (1<<23) # 8MB
        def _load(self, address, bits):
            return self.memory[address:address+(bits//8)]

        def _store(self, address, value):
            self.memory[address:address+len(value)] = value

class MAC16Option(InstructionSetOption):
    def __init__(self):
        super().__init__("mac16")
        self.registers = {
            "ACCHI": [0,0,0,0],
            "ACCLO": [0],
            "MR[0]": [0,0,0,0],
            "MR[1]": [0,0,0,0],
            "MR[2]": [0,0,0,0],
            "MR[3]": [0,0,0,0],
            "m0": "MR[0]",
            "m1": "MR[1]",
            "m2": "MR[2]",
            "m3": "MR[3]"
        }
        
        self.instructions = {
            "LDDEC": ["RRR", "Load MAC16 data register (MR) with auto decrement"],
            "LDINC": ["RRR", "Load MAC16 data register (MR) with auto increment"],
            "MUL.AA.qq": ["RRR", "Signed multiply of two address registers"],
            "MUL.AD.qq": ["RRR", "Signed multiply of an address register and a MAC16 data register"],
            "MUL.DA.qq": ["RRR", "Signed multiply of a MAC16 data register and an address register"],
            "MUL.DD.qq": ["RRR", "Signed multiply of two MAC16 data registers"],
            "MULA.AA.qq": ["RRR", "Signed multiply-accumulate of two address registers"],
            "MULA.AD.qq": ["RRR", "Signed multiply-accumulate of an address register and a MAC16 data register"],
            "MULA.DA.qq": ["RRR", "Signed multiply-accumulate of a MAC16 data register and an address register"],
            "MULA.DD.qq": ["RRR", "Signed multiply-accumulate of two MAC16 data registers"],
            "MULS.AA.qq": ["RRR", "Signed multiply/subtract of two address registers"],
            "MULS.AD.qq": ["RRR", "Signed multiply/subtract of an address register and a MAC16 data register"],
            "MULS.DA.qq": ["RRR", "Signed multiply/subtract of a MAC16 data register and an address register"],
            "MULS.DD.qq": ["RRR", "Signed multiply/subtract of two MAC16 data registers"],
            "MULA.DA.qq.LDDEC": ["RRR", "Signed multiply-accumulate of a MAC16 data register and an address register, and load a MAC16 data register with auto decrement"],
            "MULA.DA.qq.LDINC": ["RRR", "Signed multiply-accumulate of a MAC16 data register and an address register, and load a MAC16 data register with auto increment"],
            "MULA.DD.qq.LDDEC": ["RRR", "Signed multiply-accumulate of two MAC16 data registers, and load a MAC16 data register with auto decrement"],
            "MULA.DD.qq.LDINC": ["RRR", "Signed multiply-accumulate of two MAC16 data registers, and load a MAC16 data register with auto increment"],
            "UMUL.AA.qq": ["RRR", "Unsigned multiply of two address registers"],
        }

        self.expansions = {
            "qq": ["LL","HL","LH","HH"],
        }

class WindowedRegisterOption(InstructionSetOption):
    def __init__(self):
        super().__init__("windowed_register")
        self.registers = {
            "_AR0": Register(-1, "_AR0", None, 32),
            "_AR1": Register(-1, "_AR1", None, 32),
            "_AR2": Register(-1, "_AR2", None, 32),
            "_AR3": Register(-1, "_AR3", None, 32),
            "_AR4": Register(-1, "_AR4", None, 32),
            "_AR5": Register(-1, "_AR5", None, 32),
            "_AR6": Register(-1, "_AR6", None, 32),
            "_AR7": Register(-1, "_AR7", None, 32),
            "_AR8": Register(-1, "_AR8", None, 32),
            "_AR9": Register(-1, "_AR9", None, 32),
            "_AR10": Register(-1, "_AR10", None, 32),
            "_AR11": Register(-1, "_AR11", None, 32),
            "_AR12": Register(-1, "_AR12", None, 32),
            "_AR13": Register(-1, "_AR13", None, 32),
            "_AR14": Register(-1, "_AR14", None, 32),
            "_AR15": Register(-1, "_AR15", None, 32),
            "_AR16": Register(-1, "_AR16", None, 32),
            "_AR17": Register(-1, "_AR17", None, 32),
            "_AR18": Register(-1, "_AR18", None, 32),
            "_AR19": Register(-1, "_AR19", None, 32),
            "_AR20": Register(-1, "_AR20", None, 32),
            "_AR21": Register(-1, "_AR21", None, 32),
            "_AR22": Register(-1, "_AR22", None, 32),
            "_AR23": Register(-1, "_AR23", None, 32),
            "_AR24": Register(-1, "_AR24", None, 32),
            "_AR25": Register(-1, "_AR25", None, 32),
            "_AR26": Register(-1, "_AR26", None, 32),
            "_AR27": Register(-1, "_AR27", None, 32),
            "_AR28": Register(-1, "_AR28", None, 32),
            "_AR29": Register(-1, "_AR29", None, 32),
            "_AR30": Register(-1, "_AR30", None, 32),
            "_AR31": Register(-1, "_AR31", None, 32),
            "_AR32": Register(-1, "_AR32", None, 32),
            "_AR33": Register(-1, "_AR33", None, 32),
            "_AR34": Register(-1, "_AR34", None, 32),
            "_AR35": Register(-1, "_AR35", None, 32),
            "_AR36": Register(-1, "_AR36", None, 32),
            "_AR37": Register(-1, "_AR37", None, 32),
            "_AR38": Register(-1, "_AR38", None, 32),
            "_AR39": Register(-1, "_AR39", None, 32),
            "_AR40": Register(-1, "_AR40", None, 32),
            "_AR41": Register(-1, "_AR41", None, 32),
            "_AR42": Register(-1, "_AR42", None, 32),
            "_AR43": Register(-1, "_AR43", None, 32),
            "_AR44": Register(-1, "_AR44", None, 32),
            "_AR45": Register(-1, "_AR45", None, 32),
            "_AR46": Register(-1, "_AR46", None, 32),
            "_AR47": Register(-1, "_AR47", None, 32),
            "_AR48": Register(-1, "_AR48", None, 32),
            "_AR49": Register(-1, "_AR49", None, 32),
            "_AR50": Register(-1, "_AR50", None, 32),
            "_AR51": Register(-1, "_AR51", None, 32),
            "_AR52": Register(-1, "_AR52", None, 32),
            "_AR53": Register(-1, "_AR53", None, 32),
            "_AR54": Register(-1, "_AR54", None, 32),
            "_AR55": Register(-1, "_AR55", None, 32),
            "_AR56": Register(-1, "_AR56", None, 32),
            "_AR57": Register(-1, "_AR57", None, 32),
            "_AR58": Register(-1, "_AR58", None, 32),
            "_AR59": Register(-1, "_AR59", None, 32),
            "_AR60": Register(-1, "_AR60", None, 32),
            "_AR61": Register(-1, "_AR61", None, 32),
            "_AR62": Register(-1, "_AR62", None, 32),
            "_AR63": Register(-1, "_AR63", None, 32),
            "AR0": "_AR0",
            "AR1": "_AR1",
            "AR2": "_AR2",
            "AR3": "_AR3",
            "AR4": "_AR4",
            "AR5": "_AR5",
            "AR6": "_AR6",
            "AR7": "_AR7",
            "AR8": "_AR8",
            "AR9": "_AR9",
            "AR10": "_AR10",
            "AR11": "_AR11",
            "AR12": "_AR12",
            "AR13": "_AR13",
            "AR14": "_AR14",
            "AR15": "_AR15",
            "WindowBase": Register(-1, "WindowBase", "WindowBase", 8),
            "WindowStart": Register(-1,"WindowStart", "WindowStart",8),
            "PS.CALLINC": Register(-1, "PS.CALLINC", "PS.CALLINC", 8),
            "PS.OWB": Register(-1, "PS.OWB", "PS.OWB", 8),
            "PS.WOE": Register(-1, "PS.WOE", "PS.WOE", 8),
        }

        self.instructions = {
            "MOVSP": InstructionDescriptor("MOVSP at, as", InstructionFormat("RRR"), "Atomic check window and move"),
            "CALL4": InstructionDescriptor("CALL4 label", InstructionFormat("CALLX"), "Call subroutine, PC-relative, hide four registers"),
            "CALL8": InstructionDescriptor("CALL8 label", InstructionFormat("CALLX"), "Call subroutine, PC-relative, hide eight registers"),
            "CALL12": InstructionDescriptor("CALL12 label", InstructionFormat("CALLX"), "Call subroutine, PC-relative, hide twelve registers"),
            "CALLX4": InstructionDescriptor("CALLX4 as", InstructionFormat("CALLX"), "Call subroutine, address in register, hide four registers"),
            "CALLX8": InstructionDescriptor("CALLX8 as", InstructionFormat("CALLX"), "Call subroutine, address in register, hide eight registers"),
            "CALLX12": InstructionDescriptor("CALLX12 as", InstructionFormat("CALLX"), "Call subroutine, address in register, hide twelve registers"),
            "ENTRY": InstructionDescriptor("ENTRY as, 0..32760", InstructionFormat("BRI12"), "Subroutine entry, rotate registers, adjust stack pointer"),
            "RETW": InstructionDescriptor("RETW", InstructionFormat("CALLX"), "Subroutine return, unrotate registers, jump to return address"),
            "RETW.N": InstructionDescriptor("RETW.N", InstructionFormat("CALLX"), "Subroutine return, unrotate registers, jump to return address (16-bit encoding)"),
            "ROTW": InstructionDescriptor("ROTW -8..7", InstructionFormat("RRR"), "Rotate window by a constant"),
            "L32E": InstructionDescriptor("L32E at, as, -64..-4", InstructionFormat("RRI4"), "Load 32 bits for window exception"),
            "S32E": InstructionDescriptor("S32E at, as, -64..-4", InstructionFormat("RRI4"), "Store 32 bits for window exception"),
            "RFWO": InstructionDescriptor("RFWO", InstructionFormat("RRR"), "Return from window overflow exception"),
            "RFWU": InstructionDescriptor("RFWU", InstructionFormat("RRR"), "Return from window underflow exception"),
        }

    def _entry(self, imm12, s):
        assert s >= 0 and s <= 3
        
        callinc = self.find_register("PS.CALLINC").uint
        window_base = self.find_register("WindowBase").uint
        window_start = self.find_register("WindowStart").uint
        window_start |= 1 << window_base
        self.find_register("WindowBase").uint = window_base + (callinc << 2)
        self.find_register("WindowStart").uint = window_start
        
        sp_reg = "AR" + str(s)
        sp_val = self.find_register(sp_reg).uint
        self.find_register(sp_reg).uint = sp_val - imm12
        for i in range(16):
            dst = i + window_base
            self.registers["AR" + str(i)] = ("_AR" + str(dst)) if dst < 64 else None
        self.find_register(sp_reg).uint = sp_val
        a0 = self.find_register("AR0").uint
        self.find_register("AR0").uint =  (a0 & 0x3FFFFFFF) | (callinc << 30)
        return self

    def _retw(self):
        # TODO window overflow / underflow + exceptions
        pc = self.find_register("PC").uint
        ws = self.find_register("WindowStart").uint
        address = self.find_register("AR0").uint
        callinc = (address >> 30) & 0x3
        address = (address & 0x3FFFFFFF) | (pc & 0xC0000000)
        window_base = self.find_register("WindowBase").uint
        self.find_register("WindowBase").uint = window_base - (callinc << 2)
        for i in range(16):
            dst = i + window_base
            self.registers["AR" + str(i)] = ("_AR" + str(dst)) if dst < 64 else None
        if ws & (1<<window_base):
            ws &= ~(1<<window_base)
            self.find_register("WindowStart").uint =  ws
            self.find_register("PS.CALLINC").uint = callinc
            self.find_register("PC").uint = address
        
class MiscellaneousOperationsOption(InstructionSetOption):
    def __init__(self):
        super().__init__("miscellaneous")
        self.registers = {}
        self.instructions = {
            "CLAMPS": ["RRR", "Clamp to signed power of two range"],
            "MAX": ["RRR", "Max value signed"],
            "MAXU": ["RRR", "Max value unsigned"],
            "MIN": ["RRR", "Min value signed"],
            "MINU": ["RRR", "Min value signed"],
            "NSA": ["RRR", "Normalization shift amount signed"],
            "NSAU": ["RRR", "Normalization shift amount unsigned"],
            "SEXT": ["RRR", "Sign extend"]
        }

class FloatingPointCoprocessorOption(InstructionSetOption):
    def __init__(self, double_precision=True):
        super().__init__("floating_point_coprocessor")
        self.registers = {
            "FR0": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR1": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR2": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR3": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR4": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR5": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR6": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR7": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR8": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR9": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR10": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR11": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR12": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR13": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR14": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FR15": [0,0,0,0,0,0,0,0] if double_precision else [0,0,0,0],
            "FCR": [0,0,0,0],
            "FSR": [0,0,0,0],
        }

        self.instructions = {
            "ABS.p": ["RRR", "Absolute value", [0b1111, "p", "fr","fs", 0b0001, 0b0000]],
            "ADD.p": ["RRR", "Add", [0b0000, "p", "fr","fs","ft", 0b0000]],
            "ADDEXP.p": ["RRR", "Add exponent", [0b1111, "p","fr","fs",0b1110, 0b0000]],
            "ADDEXPM.p": ["RRR", "Add exponent from mantissa segment", [0b1111, "p", "fr","fs",0b1111,0b0000]],
            "CEIL.p": ["RRR", "Floating-point to signed integer conversion with round to +inf"],
            "CONST.p": ["RRR", "Create floating-point constant"],
            "DIV0.p": ["RRR", "IEEE divide initial step"],
            "DIVN.p": ["RRR", "IEEE divide final step"],
            "FLOAT.p": ["RRR", "Signed integer to floating-point conversion (current rounding mode"],
            "FLOOR.p": ["RRR", "Floating-point to signed integer conversion with round to -inf"],
            "LpI": ["RRI8", "Load immediate"],
            "LpIP": ["RRI8", "Load immediate with base post-increment"],
            "LpX": ["RRR", "Load indexed"],
            "LpXP": ["RRR", "Load with base post-increment"],
            "MADD.p": ["RRR", "Multiply-add"],
            "MADDN.p": ["RRR", "Multiply-add with round mode override to round-to-nearest"],
            "MKDADJ.p": ["RRR", "Make divide adjust amounts"],
            "MKSADJ.p": ["RRR", "Make square-root adjust amounts"],
            "MOV.p": ["RRR", "Move"],
            "MOVEQZ.p": ["RRR", "Move if equal to zero"],
            "MOVF.p": ["RRR", "Move if boolean condition false"],
            "MOVGEZ.p": ["RRR", "Move if greater than or equal to zero"],
            "MOVLTZ.p": ["RRR", "Move if less than zero"],
            "MOVNEZ.p": ["RRR", "Move if not equal to zero"],
            "MOVT.p": ["RRR", "Move if boolean condition true"],
            "MSUB.p": ["RRR", "Multiply-subtract"],
            "MUL.p": ["RRR", "Multiply"],
            "NEG.p": ["RRR", "Negate"],
            "NEXP01.p": ["RRR", "Narrow exponent"],
            "OEQ.p": ["RRR", "Compare equal"],
            "OLE.p": ["RRR", "Compare less than or equal"],
            "OLT.p": ["RRR", "Compare less than"],
            "RECIP0.p": ["RRR", "Reciprocal initial step"],
            "RFR": ["RRR", "Read floating-point register (FR to AR)"],
            "ROUND.p": ["RRR", "Floating-point to signed integer conversion with round to nearest"],
            "RSQRT0.p": ["RRR", "Reciprocal square root initial step"],
            "RUR.FCR": ["RRR", "Read floating-point control register (to AR)"],
            "RUR.FSR": ["RRR", "Read floating-point status register (to AR)"],
            "SQRT0.p": ["RRR", "Square root initial step"],
            "SSI": ["RRI8", "Store immediate"],
            "SSIP": ["RRI8", "Store immidiate with base post-increment"],
            "SSX": ["RRR", "Store indexed"],
            "SSXP": ["RRR", "Store indexed with base post-increment"],
            "SUB.p": ["RRR", "Subtract"],
            "TRUNC.p": ["RRR", "Floating-point to signed integer conversion with round to 0"],
            "UEQ.p": ["RRR", "Compare unordered or equal"],
            "UFLOAT.p": ["RRR", "Unsigned integer to floating-point conversion (current rounding mode"],
            "ULE.p": ["RRR", "Compare unordered or less than or equal"],
            "ULT.p": ["RRR", "Compare unordered or less than"],
            "UN.p": ["RRR", "Compare unordered"],
            "UTRUNC.p": ["RRR", "Floating-point to unsigned integer conversion with round to 0"],
            "WFR": ["RRR", "Write floating-point register (AR to FR)"],
            "RUR.FCR": ["RRR", "Write floating-point control register"],
            "RUR.FSR": ["RRR", "Write floating-point status register"],
            # double precision only
            "CVTD.S": ["RRR", "Convert single-precision to double-precision"],
            "CVTS.D": ["RRR", "Convert double-precision to single-precision"],
            "RFRD": ["RRR", "Read floating-point register upper (FR to AR)"],
        }

        self.expansions = {
            "p": { "S": 0b1010, "D":0b1111 } 
        }

class ESPS3ExtensionOption(InstructionSetOption):
    def __init__(self):
        super().__init__("esp32s3_extension")
        self.registers = {
            "Q0": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
            "Q1": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
            "Q2": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
            "Q3": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
            "Q4": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
            "Q5": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
            "Q6": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
            "Q7": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
            "q0": "Q0",
            "q1": "Q1",
            "q2": "Q2",
            "q3": "Q3",
            "q4": "Q4",
            "q5": "Q5",
            "q6": "Q6",
            "q7": "Q7",
            "SAR_BYTE": [0],
            "ACCX": [0,0,0,0,0],
            "QACC_H": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
            "QACC_L": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
            "FFT_BIT_WIDTH": [0],
            "UA_STATE": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        }

        self.instructions = {
            "LD.QR": ["","Load 16-byte data to QR"],
            "EE.VLD.128.p": ["", "Read the 16-byte data, then add a value to the access address."],
            "EE.VLD.hl.64.p": ["", "Read the 8-byte data, then add a value to the access address"],
            "EE.VLDBC.n32.q": [],
            "EE.VLDHBC.16.INCP": [],
            "EE.LDF.m.p": [],
            "EE.LD.128.USAR.p": [],
            "EE.LDQA.usn16.128.p": [],
            "EE.LD.QACC_hl.z.IP": [],
            "EE.LD.ACCX.IP": [],
            "EE.LD.UA_STATE.IP": [],
            "EE.LDXQ.32": [],
            "ST.QR": [],
            "EE.VST.128.p": [],
            "EE.VST.hl.64.p": [],
            "EE.STF.m.p": [],
            "EE.ST.QACC_hl.z.IP": [],
            "EE.ST.ACCX.IP": [],
            "EE.ST.UA_STATE.IP": [],
            "EE.STXQ.32": [],
            "MV.QR": [],
            "EE.MOVI.32.A": [],
            "EE.MOVI.32.Q": [],
            "EE.VZIP.n32": [],
            "EE.VUNZIP.n32": [],
            "EE.ZERO.Q": [],
            "EE.ZERO.QACC": [],
            "EE.ZERO.ACCX": [],
            "EE.MOV.usn16.QACC": [],
            "EE.VADDS.Sn32.w1": [],
            "EE.VSUBS.Sn32.w1": [],
            "EE.VMUL.usn16.w1": [],
            "EE.CMUL.S16.w1": [],
            "EE.VMULAS.usn16.ACCX.w2": [],
            "EE.VMULAS.usn16.QACC.w3": [],
            "EE.VMULAS.usn16.ACCX.w2.QUP": [],
            "EE.VMULAS.usn16.QACC.w3.QUP": [],
            "EE.SMULAS.Sn16.QACC.w4": [],
            "EE.SRCMB.Sn16.QACC": [],
            "EE.SRS.ACCX": [],
            "EE.VRELU.Sn8": [],
            "EE.VPRELU.Sn16": [],
            "EE.VMAX.Sn32.w1": [],
            "EE.VMIN.Sn32.w1": [],
            "EE.VCMP.ee.Sn32": [],
            "EE.ORQ": [],
            "EE.XORQ": [],
            "EE.ANDQ": [],
            "EE.NOTQ": [],
            "EE.SRC.Q": [],
            "EE.SRC.Q.QUP": [],
            "EE.SRC.Q.LD.p": [],
            "EE.SLCI.2Q": [],
            "EE.SLCXXP.2Q": [],
            "EE.SRCI.2Q": [],
            "EE.SRCXXP.2Q": [],
            "EE.SRCQ.128.ST.INCP": [],
            "EE.VSR.32": [],
            "EE.VSL.32": [],
            "EE.FFT.R2BF.S16": [],
            "EE.FFT.R2BF.S16.ST.INCP": [],
            "EE.FFT.CMUL.S16.w5": [],
            "EE.BITREV": [],
            "EE.FFT.AMS.S16.w6": [],
            "EE.FFT.VST.R32.DECP": [],
            "EE.WR_MASK_GPIO_OUT": [],
            "EE.SET_BIT_GPIO_OUT": [],
            "EE.CLR_BIT_GPIO_OUT": [],
            "EE.GET_GPIO_IN": [],
        }

        self.expansions = {
            "p": { "IP": 1, "XP":2 },
            "q": { "": 0, "IP": 1, "XP": 2 },
            "n16": { "8": 3, "16": 4 },
            "n32": { "8": 3, "16": 4, "32": 5 },
            "m": { "64": 0, "128": 1 },
            "us": { "U": 0x0000, "S": 0x0001 },
            "hl": { "H": 0b0000, "L": 0b0001 },
            "z": {"H.32": 0, "L.128":1 },
            "w1": { "": 0, "LD.INCP": 1, "ST.INCP": 0 },
            "w2": { "": 0, "LD.IP": 1, "LD.XP": 2},
            "w3": { "": 0, "LD.IP": 1, "LD.XP": 2, "LDBC.INCP":3},
            "w4": { "": 0, "LD.INCP": 1},
            "w5": { "LD.XP": 0, "ST.XP": 1 },
            "w6": { "LD.INCP.UAUP":0, "LD.INCP": 1, "LD.R32.DECP": 2, "ST.INCP": 3 },
            "ee": { "EQ": 0, "LT": 1, "GT": 2 },
        }
        
core = CoreInstructionSet()
core.extend(MAC16Option())
core.extend(WindowedRegisterOption())

instruction = core.find_instruction("L16SI")
print(instruction.template)

instruction = Instruction.parse(core, "BBCI a2, 24, my_label")
core.find_register("PC").uint = 128
core.find_register("AR3").uint = 1024
core.find_register("AR12").uint = 13
core.find_register("AR0").uint = 132

print("AR0", core.find_register("AR0").uint)
print("AR3", core.find_register("AR3").uint)

core.find_register("PS.CALLINC").uint = 2
core.options["windowed_register"]._entry(256, 3)
core.find_register("AR2").uint = 21

print("AR0 = 21?", core.find_register("AR0"))
print("AR3 = 768?", core.find_register("AR3"))
print("AR4 = 13?", core.find_register("AR4"))

core.options["windowed_register"]._retw()
print("AR0 = 132?", core.find_register("AR0").uint)
print("AR3 = 1024?", core.find_register("AR3").uint)
print("AR10 = 21?", core.find_register("AR10").uint)
print("AR12 = 13?", core.find_register("AR12").uint)


WARNING. <__main__.WindowedRegisterOption object at 0x7607e7f5b3d0> overwrites register AR0
WARNING. <__main__.WindowedRegisterOption object at 0x7607e7f5b3d0> overwrites register AR1
WARNING. <__main__.WindowedRegisterOption object at 0x7607e7f5b3d0> overwrites register AR2
WARNING. <__main__.WindowedRegisterOption object at 0x7607e7f5b3d0> overwrites register AR3
WARNING. <__main__.WindowedRegisterOption object at 0x7607e7f5b3d0> overwrites register AR4
WARNING. <__main__.WindowedRegisterOption object at 0x7607e7f5b3d0> overwrites register AR5
WARNING. <__main__.WindowedRegisterOption object at 0x7607e7f5b3d0> overwrites register AR6
WARNING. <__main__.WindowedRegisterOption object at 0x7607e7f5b3d0> overwrites register AR7
WARNING. <__main__.WindowedRegisterOption object at 0x7607e7f5b3d0> overwrites register AR8
WARNING. <__main__.WindowedRegisterOption object at 0x7607e7f5b3d0> overwrites register AR9
WARNING. <__main__.WindowedRegisterOption object at 0x7607e7f5b3d0> overwrites r